In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [3]:
random_state = 114
criterion = "entropy"
max_depth = 8
max_features = "sqrt"
n_estimators = 150
max_leaf_nodes = 90
bootstrap = True

data_table = pd.read_csv('data/german.csv')
#for feature in unimportance_features:
#    data_table = data_table.drop(feature, axis=1)
X = data_table.drop('Creditability', axis=1).values
y = data_table['Creditability'].values

sm = SMOTE(random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=random_state)
X_train, y_train = sm.fit_resample(X_train, y_train)


best_acc = 0

clf = RandomForestClassifier(
    max_leaf_nodes=max_leaf_nodes,
    max_features=max_features,
    bootstrap=bootstrap,
    criterion=criterion, 
    max_depth=max_depth,
    random_state=random_state,
    n_estimators=n_estimators,
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
features = data_table.drop('Creditability', axis=1).columns.to_list()
acc = accuracy_score(y_test, y_pred)

print('Accuracy Score is', accuracy_score(y_test, y_pred))
print('Precision is', precision_score(y_test, y_pred))
print('Recall is', recall_score(y_test, y_pred))
print('F1-Score is', f1_score(y_test, y_pred))

Accuracy Score is 0.82
Precision is 0.8820754716981132
Recall is 0.8657407407407407
F1-Score is 0.8738317757009345


In [4]:

class PathExtractor():
    def __init__(self, forest, X, y, X_train, y_train, features):
        self.X, self.y = X, y   # original training data

        self.features = features
        self.n_features = len(self.features)
        self.categories = np.unique(y).tolist()
        self.n_categories = len(self.categories)
        self.feature_range = [np.min(X, axis=0), np.max(X, axis=0)+1e-9]
        self.category_total = [np.sum(self.y == i)
                               for i in range(self.n_categories)]
        self.forest = forest
        self.X = X
        self.y = y
        self.X_train = X_train
        self.y_train = y_train
        self.n_examples = len(self.y)
        self.n_examples2 = len(self.y_train)
        self.n_estimators = forest.n_estimators

    def get_paths(self, min_impurity_decrease=0.0):
        self.paths = [[] for i in range(self.n_estimators)]
        self.min_impurity_decrease = min_impurity_decrease
        for i in range(self.n_estimators):
            self.dfs(i, 0, {}, np.ones(self.n_examples),
                     np.ones(self.n_examples2))
        return self.paths

    def dfs(self, i, u, feature_range, vec_examples, vec_examples2):
        tr = self.forest.estimators_[i].tree_

        def impurity_decrease(tr, u):
            N_t = tr.n_node_samples[u]
            I_t = tr.impurity[u]
            N = tr.n_node_samples[0]
            Lc = tr.children_left[u]
            Rc = tr.children_right[u]
            N_l = tr.n_node_samples[Lc]
            I_l = tr.impurity[Lc]
            N_r = tr.n_node_samples[Rc]
            I_r = tr.impurity[Rc]
            return N_t/N*(I_t-N_r/N_t*I_r-N_l/N_t*I_l)

        def cpy(m):
            return {key: m[key].copy() for key in m}

        if tr.children_left[u] < 0 or tr.children_right[u] < 0 or impurity_decrease(tr, u) < self.min_impurity_decrease:
            distribution = [np.dot(vec_examples, self.y == cid)
                            for cid in range(self.n_categories)]
            distribution2 = [np.dot(vec_examples2, self.y_train == cid)
                             for cid in range(self.n_categories)]
            output = np.argmax(distribution2)
            coverage = sum(distribution)
            if coverage > 0:
                self.paths[i].append({
                    "name": 'r%d-%d' % (len(self.paths[i]), i),
                    "tree_index": i,
                    "rule_index": len(self.paths[i]),
                    "range": {str(key): feature_range[key].copy() for key in feature_range},
                    "distribution": distribution,
                    "coverage": coverage,
                    "fidelity": distribution[int(output)] / coverage,
                    "sample": vec_examples,
                    "output": str(output)
                })
        else:
            feature = tr.feature[u]
            threshold = tr.threshold[u]

            _feature_range = cpy(feature_range)
            if not feature in feature_range:
                _feature_range[feature] = [self.feature_range[0]
                                           [feature], self.feature_range[1][feature]+1e-9]
            _feature_range[feature][1] = min(
                _feature_range[feature][1], threshold)

            _vec_examples = vec_examples*(self.X[:, feature] <= threshold)
            _vec_examples2 = vec_examples2 * \
                (self.X_train[:, feature] <= threshold)

            self.dfs(
                i, tr.children_left[u], _feature_range, _vec_examples, _vec_examples2)

            _feature_range = cpy(feature_range)
            if not feature in feature_range:
                _feature_range[feature] = [self.feature_range[0]
                                           [feature], self.feature_range[1][feature]]
            _feature_range[feature][0] = threshold

            _vec_examples = vec_examples*(self.X[:, feature] > threshold)
            _vec_examples2 = vec_examples2 * \
                (self.X_train[:, feature] > threshold)
            self.dfs(
                i, tr.children_right[u], _feature_range, _vec_examples, _vec_examples2)

    # given X as input, find the range of fid-th feature to keep the prediction unchanged
    def getRange(self, X, fid):
        step = (self.feature_range[1][fid]-self.feature_range[0][fid])*0.005
        L, R = X[fid], X[fid]
        Xi = X.copy()
        ei = np.array([1 if i == fid else 0 for i in range(self.n_features)])
        result0 = self.predict([X])[0]
        result1 = result0

        while(result1 == result0 and L > self.feature_range[0][fid]):
            Xi = Xi-step*ei
            result1 = self.predict([Xi])[0]
            L -= step
        L = max(L, self.feature_range[0][fid])
        LC = result1

        Xi = X.copy()
        while(result1 == result0 and R < self.feature_range[1][fid]):
            Xi = Xi+step*ei
            result1 = self.predict([Xi])[0]
            R += step
        R = min(R, self.feature_range[1][fid])
        RC = result1
        return {
            "L": L,
            "LC": LC,  # the prediction when X[fid]=L-eps
            "R": R,
            "RC": RC,  # the prediction when X[fid]=R+eps
        }

rf = PathExtractor(clf, X, y, X_train, y_train, features)

In [5]:
paths = rf.get_paths()
paths = [p for r in paths for p in r]
print(len(paths))

10733


In [6]:
import pulp
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

class Extractor:
    # 可以调用的接口：compute_accuracy和extract
    def __init__(self, rf, X_raw, y_raw):
        # rf：随机森林模型
        # X_raw、y_raw：训练数据集
        self.rf = rf
        self.X_raw = X_raw
        self.y_raw = y_raw
        _paths = rf.get_paths()
        self.paths = [p for r in _paths for p in r]# if p['fidelity'] > 0.7]
        self.paths.sort(key=lambda x: -x['coverage'])

    def compute_accuracy(self, paths):
        # 计算数据集在给定规则集下的accuracy
        # paths：规则集，为list
        Mat = self.getMat(self.X_raw, self.y_raw, paths)
        idx = np.argwhere(np.all(Mat[..., :] == 0, axis=0))
        Mat = np.delete(Mat, idx, axis=1)
        right = np.sum(Mat, axis=0)
        return np.sum(np.where(right >= 0, 1, 0)) / len(self.X_raw)

    def predict(self, paths):
        # 计算数据集在给定规则集下的accuracy
        # paths：规则集，为list
        Mat = self.getMat(self.X_raw, self.y_raw, paths)
        idx = np.argwhere(np.all(Mat[..., :] == 0, axis=0))
        Mat = np.delete(Mat, idx, axis=1)
        right = np.sum(Mat, axis=0)
        return np.where(right > 0, 1, 0)

    def extract(self, max_num, tau):
        # 根据给定的max_num和tau，使用rf的全部规则和数据集抽取出相应的规则
        # max_num：抽取出规则的最大数量
        # tau：每个样本允许的最大惩罚
        # 返回抽取出规则的列表、数据集使用全部规则的accuracy、数据集使用抽取规则的accuracy
        Mat = self.getMat(self.X_raw, self.y_raw, self.paths)
        self.Mat = Mat
        w = self.getWeight(Mat)
        new_paths, new_path_indexes = self.LP_extraction(w, Mat, max_num, tau)
        accuracy_origin = self.compute_accuracy(self.paths)
        accuracy_new = self.compute_accuracy(new_paths)
        return new_path_indexes, new_paths, accuracy_origin, accuracy_new

    def path_score(self, path, X, y):
        ans = 2 * (y == int(path.get('output'))) - 1
        m = path.get('range')
        for key in m:
            ans = ans * (X[:, int(key)] >= m[key][0]) * (X[:, int(key)] < m[key][1])
        return ans

    def getMat(self, X_raw, y_raw, paths):
        # 覆盖矩阵Mat
        Mat = np.array([self.path_score(p, X_raw, y_raw) for p in paths]).astype('float')
        return Mat

    def getWeight(self, Mat):
        # 权重向量w
        RXMat = np.abs(Mat)
        XRMat = RXMat.transpose()
        XXAnd = np.dot(XRMat, RXMat)
        XROne = np.ones(XRMat.shape)
        XXOr = 2 * np.dot(XROne, RXMat) - XXAnd
        XXOr = (XXOr + XXOr.transpose()) / 2
        XXDis = 1 - XXAnd / XXOr
        K = int(np.ceil(np.sqrt(len(self.X_raw))))
        clf = LocalOutlierFactor(n_neighbors=K, metric="precomputed")
        clf.fit(XXDis)
        XW = -clf.negative_outlier_factor_
        MXW, mXW = np.max(XW), np.min(XW)
        XW = 1 + (3 - 1) * (XW - mXW) / (MXW - mXW)
        return XW / np.sum(XW)

    def LP_extraction(self, w, Mat, max_num, tau):
        m = pulp.LpProblem(sense=pulp.LpMinimize)
        # 创建最小化问题
        var = []
        for i in range(len(self.paths)):
            var.append(pulp.LpVariable(f'x{i}', cat=pulp.LpBinary))
        for i in range(len(w)):
            var.append(pulp.LpVariable(f'k{i}', cat=pulp.LpInteger, lowBound=0))
        # 添加变量x_0至x_{M-1}, k_0至k_{N-1}

        m += pulp.lpSum([w[j] * (var[j + len(self.paths)])
                         for j in range(len(w))])
        # 添加目标函数

        m += (pulp.lpSum([var[j] for j in range(len(self.paths))]) <= max_num)
        # 筛选出不超过max_num条规则

        for j in range(len(w)):
            m += (var[j + len(self.paths)] >= 1000 + tau - pulp.lpSum([var[k] * Mat[k][j] for k in range(len(self.paths))]))
            m += (var[j + len(self.paths)] >= 1000)
            # max约束

        m.solve(pulp.PULP_CBC_CMD())#solver = pulp.solver.CPLEX())#
        new_paths = [self.paths[i] for i in range(len(self.paths)) if var[i].value() > 0]
        new_path_indexes = [self.paths[i]['name'] for i in range(len(self.paths)) if var[i].value() > 0]
        return new_paths, new_path_indexes


In [8]:
ex = Extractor(rf, X_train, y_train)
ex.compute_accuracy(paths)

0.8822314049586777

In [11]:
label1 = ex.predict(paths)
y_train2 = clf.predict(X_train)
label2 = y_train == y_train2


In [31]:
def check_sample(paths, X):
    ret = []
    for path in paths:
        m = path['range']
        ans = 1
        for key in m:
            ans = ans * (X[int(key)] >= m[key][0]) * (X[int(key)] < m[key][1])
        if ans > 0:
            ret.append(path['output'])
    return ret

In [63]:
index = 18
ret = check_sample(paths, X_train[index])
print(len(ret), sum([int(x) for x in ret]))
clf.predict([X_train[index]])

150 67


array([1])

In [55]:
y_train

array([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,

train_y

In [168]:
ret = ex.extract(75, 10)


In [171]:
print(ret[2:])
paths = rf.get_paths()

all_paths = []
for t in paths:
    all_paths = all_paths + t

import shap

explainer = shap.Explainer(r_clf)
shap_values = explainer(X)

(0.892, 0.804)


In [185]:
features = []
for index, feature in enumerate(data.columns):
    if len(feature_aggregate[index]) == 1:
        i = feature_aggregate[index][0]
        features.append({
            "name": feature,
            "range": [rf.feature_range[0][i], rf.feature_range[1][i]],
            "importance": r_clf.feature_importances_[i],
            "dtype": "numeric",
        })
    else:
        features.append({
            "name": feature,
            "range": [feature_unique[feature]],
            "importance": sum([r_clf.feature_importances_[i] for i in feature_aggregate[index]]),
            "dtype": "object",
        })

for path in all_paths:
    new_range = {}
    for index in path['range']:
        feature = feature_origin[int(index)]
        if type(feature) == list:
            if feature[0] not in new_range:
                new_range[feature[0]] = [-1] * len(feature_unique[feature[0]])
            new_range[feature[0]][feature[1]] = 0 if path['range'][index][0] == 0 else 1
        else:
            new_range[feature] = path['range'][index]
    path['range'] = new_range

output_data = {
    'paths': all_paths,
    'features': features,
    'selected': ret[0],
    'shap_values': shap_values,
}

import pickle
pickle.dump(output_data, open('output/german2.pkl', 'wb'))

In [187]:
all_paths[5]

{'name': 'r5-0',
 'tree_index': 0,
 'rule_index': 5,
 'range': {'Length of current employment': [-1, -1, 0, 0, -1],
  'Account Balance': [-1, -1, 0, 0],
  'Payment Status of Previous Credit': [-1, -1, 1, -1, 0],
  'Value Savings/Stocks': [-1, -1, 0, -1, -1],
  'Credit Amount': [669.5, 18424.000000001],
  'Purpose': [-1, -1, -1, -1, 1, -1, -1, -1, -1, -1]},
 'distribution': [22.0, 14.0],
 'coverage': 36.0,
 'fidelity': 0.6111111111111112,
 'sample': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 

In [56]:
features

['A11',
 'A12',
 'A13',
 'A14',
 'Duration of Credit (month)',
 'A30',
 'A31',
 'A32',
 'A33',
 'A34',
 'A40',
 'A41',
 'A410',
 'A42',
 'A43',
 'A44',
 'A45',
 'A46',
 'A48',
 'A49',
 'Credit Amount',
 'A61',
 'A62',
 'A63',
 'A64',
 'A65',
 'A71',
 'A72',
 'A73',
 'A74',
 'A75',
 'Installment per cent',
 'A91',
 'A92',
 'A93',
 'A94',
 'A101',
 'A102',
 'A103',
 'Duration in Current address',
 'A121',
 'A122',
 'A123',
 'A124',
 'Age (years)',
 'A141',
 'A142',
 'A143',
 'A151',
 'A152',
 'A153',
 'No of Credits at this Bank',
 'A171',
 'A172',
 'A173',
 'A174',
 'No of dependents',
 'A191',
 'A192',
 'A201',
 'A202']

In [126]:
sum(ex.Mat[10] == 1)

165

In [21]:
print(shap_values[0])

.values =
array([[ 1.09110125e-01, -1.09110125e-01],
       [ 3.01264698e-03, -3.01264698e-03],
       [-1.24874332e-01,  1.24874332e-01],
       [ 2.10891519e-03, -2.10891519e-03],
       [-1.99799022e-02,  1.99799022e-02],
       [ 3.40283799e-02, -3.40283799e-02],
       [ 3.85628726e-02, -3.85628726e-02],
       [ 5.60976830e-03, -5.60976830e-03],
       [ 3.36153339e-02, -3.36153339e-02],
       [ 3.85308858e-03, -3.85308858e-03],
       [-2.39386825e-02,  2.39386825e-02],
       [ 7.02421386e-03, -7.02421386e-03],
       [ 1.96232689e-02, -1.96232689e-02],
       [-1.74483971e-02,  1.74483971e-02],
       [ 6.65478065e-02, -6.65478065e-02],
       [ 6.73321595e-03, -6.73321595e-03],
       [ 1.58162875e-03, -1.58162875e-03],
       [-6.93273863e-05,  6.93273863e-05],
       [ 5.12965848e-03, -5.12965848e-03],
       [ 2.51444887e-03, -2.51444887e-03]])

.base_values =
array([0.50028956, 0.49971044])

.data =
array([   1,   18,    4,    2, 1049,    1,    2,    4,    2,    1,    4,

In [155]:
left_paths = [path for path in all_paths if path['fidelity'] > 0.75]
left_paths.sort(key=lambda x: -x['coverage'])

In [53]:
import random
print(ex.compute_accuracy([ex.paths[i] for i in random.sample(range(3000), 75)]))

0.76


In [158]:
print(ret[])

{'name': 'r67-90',
 'tree_index': 90,
 'rule_index': 67,
 'range': {'3': [0.5, 1.000000001],
  '8': [0, 0.5],
  '44': [19, 66.5],
  '49': [0.5, 1.000000001]},
 'distribution': [0.0, 245.0],
 'coverage': 245.0,
 'fidelity': 1.0,
 'sample': array([0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
  